<a href="https://colab.research.google.com/github/AdamKimhub/Msproject1/blob/colab/forfinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if "google.colab" in sys.modules:
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    dataset_path = '/content/drive/My Drive/dataset'

    # Install required packages
    !pip install pymatgen torch torch_geometric

else:
    dataset_path = 'dataset'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB

In [2]:
from pathlib import Path
import ast
import numpy as np
import pandas as pd
from pymatgen.core import Structure, PeriodicSite, DummySpecie
from pymatgen.analysis.local_env import MinimumDistanceNN


## When  dealing with high dataset only

In [ ]:
high_dataset = ["high_BN", "high_GaSe", "high_InSe", "high_MoS2", "high_P", "high_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{high_data}.csv") for high_data in high_dataset]

high_df  = pd.concat(to_merge, ignore_index=True)

high_copy = high_df.copy()

In [ ]:
high_copy = high_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
high_copy = high_copy.corr()
high_copy

## When dealing with low dataset

In [ ]:
low_dataset = ["low_MoS2", "low_WSe2"]
to_merge = [pd.read_csv(f"{dataset_path}/combined/{low_data}.csv") for low_data in low_dataset]

low_df  = pd.concat(to_merge, ignore_index=True)

low_copy = low_df.copy()

In [ ]:
low_copy = low_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
low_copy = low_copy.corr()
low_copy

## When dealing with both low and high dataset

In [ ]:
# Read file
sample_df = pd.read_csv(f"{dataset_path}/combined/combined.csv")

In [ ]:
sample_copy = sample_df.copy()
sample_copy = sample_copy.drop(["_id", "base", "cell", "dataset_material", "fermi_level", "total_mag"], axis =1)
sample_copy = sample_copy.corr()
sample_copy

## Data to graphs

In [25]:
samplex_df = pd.read_csv(f"{dataset_path}/combined/combined.csv")

In [27]:
import to_graph
import torch
from torch_geometric.data import Data

In [28]:

def graphy(row):
    defective_file_path = Path(f"{dataset_path}/{row['dataset_material']}/cifs/{row['_id']}.cif")
    defective_structure = Structure.from_file(defective_file_path)

    ref_file_path = Path(f"{dataset_path}/{row['dataset_material']}/{row['base']}.cif")
    ref_unit_cell = Structure.from_file(ref_file_path)
    the_cell = ast.literal_eval(row['cell'])
    reference_structure = ref_unit_cell.make_supercell(the_cell)

    defects_structure = to_graph.get_defects_structure(defective_structure, reference_structure)

    the_nodes, the_edges, the_edge_features = to_graph.get_nodes_edges(defects_structure)

    global_attributes = ["energy","fermi_level","total_mag","formation_energy",
                         "energy_per_atom","E_1","vacancy_sites",
                         "substitution_sites","defect_sites"]

    global_features = [row[i] for i in global_attributes]

    target_attribute = "band_gap_value"
    target_features = [row[target_attribute]]

    the_data = Data(x=torch.tensor(the_nodes), edge_index=torch.tensor(the_edges),
                    edge_attr=torch.tensor(the_edge_features),
                    u=torch.tensor(global_features), y=torch.tensor(target_features))
    return the_data

samplex = samplex_df.apply(lambda row: graphy(row), axis = 1).tolist()

# Save the data
torch.save(samplex, f"{dataset_path}/combined/all_graphs.pt")

/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 32 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 31 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]
/usr/local/lib/python3.11/dist-packages/pymatgen/core/structure.py:3102: UserWarning: Issues encountered while parsing CIF: 30 fractional coordinates rounded to ideal values to avoid is

KeyboardInterrupt: 

## Build model with the graph data

In [19]:
# Load the data
whole_graph = torch.load(f"{dataset_path}/combined/all_graphs.pt", weights_only=False)

In [ ]:
# Split the data
from sklearn.model_selection import train_test_split

train_graphs, test_graphs = train_test_split(whole_graph, test_size=0.3, stratify="dataset_type", random_state=42)

val_graphs, test_graphs = train_test_split(test_graphs, test_size=0.5, stratify="dataset_type", random_state=42)

In [ ]:
# Create data loaders
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)
val_loader = DataLoader(val_graphs, batch_size=32, shuffle=False)
test_loader = DataLoader(test_graphs, batch_size=32, shuffle=False)

In [ ]:
new_df = torch.# samplex[0]    return the_nodes# samplex[0]    return the_nodes, the_edges, the_edge_features, global_features, target_features, the_edges, the_edge_features, global_features, target_features